In [1]:
from scytheq.data import Databank
from scytheq.backtest import *
from scytheq.analysis import *

ModuleNotFoundError: No module named 'scytheq'

In [6]:
data = Databank()

#### Classic

In [6]:
ROE = (data['常續ROE']).to_rank()
MTM = (data['收盤價']*data['調整係數']).pct_change(60).to_rank()
PBR = (-1*data['股價淨值比']).to_rank()
classic_tw3factor = backtesting(
    (ROE+PBR+MTM).to_rank()>=0.99, # 等權三因子, 並選前 1% 投資
    rebalance='QR', # 每一季底再平衡 (使用最近一期的財報資料)
    start='2008',
    report=True,
)

,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),25.77,9.08,14.59
Sharpe,1.19,0.57,0.95
Calmar,0.44,0.17,0.57
MDD(%),-59.20,-53.64,-25.71
Simple MDD(%),-64.62,-65.22,-22.45
Win Rate(%),57.69,54.74,53.50
Weekly Win(%),59.78,56.15,55.05
Monthly Win(%),65.24,62.38,55.71
Yearly Win(%),66.67,66.67,72.22
Win/Loss Ratio,0.91,0.92,1.02


In [ ]:
classic_tw3factor.tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'530d4ee5-9f01-4560-a189-b1d5273ec466': {'version…

#### Profit

In [18]:
profit_factors = {
    'ROE': data['常續ROE'],
    'ROE_Q': (data['常續ROE'] - data['常續ROE_Lag1']),
    'ROE_Y': (data['常續ROE'] - data['常續ROE_Lag4']),
}

profit_quantiles = pd.concat(
    [
        calc_metrics(pd.DataFrame(calc_factor_quantiles_return(v.to_rank(), rebalance='QR', start='2008', add_ls=True)))\
            .loc['CAGR(%)']\
            .rename(k) 
            for k, v in profit_factors.items()
    ], 
    axis=1
)
display(profit_quantiles)

100%|██████████| 10/10 [00:01<00:00,  7.12it/s]


,ROE,ROE_Q,ROE_Y
0.0,1.07,2.00,-1.20
0.1,6.65,4.19,3.18
0.2,8.85,7.30,5.71
0.3,8.27,7.90,8.03
0.4,8.98,8.89,10.64
0.5,12.01,12.36,12.58
0.6,12.70,13.10,13.60
0.7,13.23,14.22,15.34
0.8,14.68,16.14,18.26
0.9,14.66,16.06,17.34


#### Value

In [23]:
value_factors = {
    'PER':(data['本益比']*-1),
    'PBR':(data['股價淨值比']*-1),
    'PCR':(data['收盤價']/data['營運產生現金流量']*-1),
    'MCAP_TO_REV':(data['個股市值(元)']/data['營業收入']*-1),
}

value_quantiles = pd.concat(
    [
        calc_metrics(pd.DataFrame(calc_factor_quantiles_return(v.to_rank(), rebalance='QR', start='2008', add_ls=True)))\
            .loc['CAGR(%)']\
            .rename(k) 
            for k, v in value_factors.items()
    ], 
    axis=1
)
display(value_quantiles)

100%|██████████| 10/10 [00:01<00:00,  5.15it/s]


,PER,PBR,PCR,MCAP_TO_REV
0.0,6.37,2.17,8.20,4.67
0.1,7.44,5.37,12.04,5.59
0.2,8.37,7.53,11.56,8.96
0.3,9.32,8.67,10.88,8.69
0.4,9.56,9.18,13.84,9.51
0.5,11.93,11.06,12.02,10.40
0.6,12.14,11.66,9.52,11.95
0.7,13.06,12.53,7.56,11.90
0.8,12.87,15.21,8.65,13.10
0.9,9.24,16.27,6.44,16.35


#### Momentum

In [22]:
returns = data['報酬率']
turnover = data['周轉率']
days = 20
FIP = (
    returns.apply(lambda x: ((x>0).rolling(days).sum())).to_factor() -
    turnover.rolling(days).mean().to_factor()
).to_rank()

In [25]:
mtm_factors = {
    'MTM_3m':(data['收盤價']*data['調整係數']).pct_change(60),
    'FIP':FIP,
}

mtm_quantiles = pd.concat(
    [
        calc_metrics(pd.DataFrame(calc_factor_quantiles_return(v.to_rank(), rebalance='QR', start='2008', add_ls=True)))\
            .loc['CAGR(%)']\
            .rename(k) 
            for k, v in mtm_factors.items()
    ], 
    axis=1
)
display(mtm_quantiles)

100%|██████████| 10/10 [00:01<00:00,  8.74it/s]


,MTM_3m,FIP
0.0,6.16,3.49
0.1,4.69,9.42
0.2,7.40,9.08
0.3,7.21,8.96
0.4,8.84,9.57
0.5,10.04,11.51
0.6,9.84,12.28
0.7,10.85,12.96
0.8,12.77,14.09
0.9,14.30,16.73


#### Parameters

In [36]:
ROE_Y = (data['常續ROE']-data['常續ROE_lag4']).to_rank()
MCAP_TO_REV = (data['個股市值(元)']/data['營業收入']*-1).to_rank()
FIP = (
    returns.apply(lambda x: ((x>0).rolling(days).sum())).to_factor() -
    turnover.rolling(days).mean().to_factor()
).to_rank()
TW3F = (ROE_Y+MCAP_TO_REV+FIP).to_rank()
tw3f_report2 = backtesting(
   TW3F>=0.99, # 等權三因子
   rebalance='QR', 
   start=2008,
   report=True, 
)

,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),35.53,9.08,22.49
Sharpe,1.84,0.57,1.34
Calmar,0.74,0.17,0.87
MDD(%),-48.17,-53.64,-25.77
Simple MDD(%),-54.77,-65.22,-15.79
Win Rate(%),57.38,54.74,53.25
Weekly Win(%),60.00,56.15,55.16
Monthly Win(%),66.67,62.38,65.24
Yearly Win(%),83.33,66.67,94.44
Win/Loss Ratio,1.03,0.92,1.10


In [30]:
close = data['收盤價']
TW3F = (ROE_Y+MCAP_TO_REV+FIP).to_rank(universe=(close>=close.rolling(60).mean()))
tw3f_report3 = backtesting(
   TW3F>=0.99, # 等權三因子
   rebalance='QR', 
   start=2008,
   report=True, 
)

,Strategy,Benchmark: TRTEJ,Excess Return
CAGR(%),44.48,9.08,30.47
Sharpe,1.61,0.57,1.22
Calmar,1.13,0.17,0.76
MDD(%),-39.36,-53.64,-40.21
Simple MDD(%),-26.06,-65.22,-15.04
Win Rate(%),53.29,54.74,51.82
Weekly Win(%),58.68,56.15,54.62
Monthly Win(%),65.71,62.38,58.10
Yearly Win(%),88.89,66.67,77.78
Win/Loss Ratio,1.14,0.92,1.15


In [32]:
tw3f_report3.tabs

BokehModel(combine_events=True, render_bundle={'docs_json': {'6b60ce21-bf3c-49f0-8b29-d79a75c3eab4': {'version…

In [43]:
pd.concat(
    [
        classic_tw3factor.metrics['Strategy'].rename('Original'),
        tw3f_report2.metrics['Strategy'].rename('Improved'),
        tw3f_report3.metrics['Strategy'].rename('Ultimate'),
    ],
    axis=1
)

,Original,Improved,Ultimate
CAGR(%),25.77,35.53,44.48
Sharpe,1.19,1.84,1.61
Calmar,0.44,0.74,1.13
MDD(%),-59.20,-48.17,-39.36
Simple MDD(%),-64.62,-54.77,-26.06
Win Rate(%),57.69,57.38,53.29
Weekly Win(%),59.78,60.00,58.68
Monthly Win(%),65.24,66.67,65.71
Yearly Win(%),66.67,83.33,88.89
Win/Loss Ratio,0.91,1.03,1.14
